### Setup

In [1]:
import os, sys
sys.path.append('..')

import warnings
warnings.filterwarnings('ignore', category=UserWarning, module='cobra')

from cobra import io, Model, Solution
from scripts.helpers.model import rxn_in_model, met_in_model, add_single_gene_reaction_pair
from scripts.opt._fba import flux_balance_analysis
from scripts.opt._fva import run_flux_variability_analysis

In [2]:
# Gap-filled model
model = io.read_sbml_model('../data/altered/xmls/MNL_iCre1355_auto_GAPFILL/SQS+SQE+MVA.xml')

# Add exchange reaction
ERG = "ergosterol_c"
ERGEXCH = "ERGOSTEROLEXCH"
if not rxn_in_model(model, ERGEXCH):
    add_single_gene_reaction_pair(
        model=model, 
        gene_id="EXCHERG_GENE",
        reaction_id=ERGEXCH,
        reaction_name="Ergosterol exchange (assumption)", 
        reaction_subsystem="Exchange", 
        metabolites=[(-1, ERG)],
        reversible=True
    )

No objective coefficients in model. Unclear what should be optimized


### FVA (Blocked)

In [3]:
fva = run_flux_variability_analysis(
    model,
    loopless=True,
    pfba_factor=None,
    objectives=['ALT_MVAS'],
    reactions=['ALT_MVAS', 'ALT_MVAD', 'ALT_MVAE', 'ALT_PMK', 'ALT_MVK', 'ALT_IDLI']
)

fva

,minimum,maximum
ALT_MVAS,0.0,1.333895e-34
ALT_MVAD,0.0,0.000000e+00
ALT_MVAE,0.0,1.333895e-34
ALT_PMK,0.0,3.381919e-32
ALT_MVK,0.0,1.333895e-34
ALT_IDLI,0.0,1.379310e-01


### FVA (Repaired)

In [20]:
temp = model.copy()

# Acetyl_CoA_c = temp.metabolites.get_by_id("accoa_c")
# Acetyl_CoA_h = temp.metabolites.get_by_id("accoa_h")

FPPS = temp.reactions.get_by_id('FPPS')
mets = FPPS.metabolites

add_single_gene_reaction_pair(
    model=temp,
    gene_id="FPPS_ATTEMPT",
    reaction_id='FPPS_NEW',
    reaction_name="Farnesyl pyrophosphate synthase (manual)",
    reaction_subsystem="Biosynthesis of steroids",
    metabolites=[ (k, v.id) for v, k in mets.items() ],
    reversible=False
)

FPPS.lower_bound = 0
FPPS.upper_bound = 0

fva = run_flux_variability_analysis(
    model=temp,
    loopless=True,
    pfba_factor=None,
    objectives=['ALT_MVAS'],
    reactions=['ALT_MVAS', 'ALT_MVAD', 'ALT_MVAE', 'ALT_PMK', 'ALT_MVK', 'ALT_IDLI']
)

fva

,minimum,maximum
ALT_MVAS,0.0,1.333895e-34
ALT_MVAD,0.0,0.000000e+00
ALT_MVAE,0.0,1.333895e-34
ALT_PMK,0.0,3.381919e-32
ALT_MVK,0.0,1.333895e-34
ALT_IDLI,0.0,1.379310e-01


### FVA (Sterol)